In [1]:
import os
import zarr
import xarray
import rasterio
import rioxarray
import numpy as np
import datetime as dt

patches_dir = "./Data/Images/Patches"
data_cube_dir = "./Data/Images/Data Cubes"
cube_log = "./Log/Data Cubes Log.txt"
walkthrough = os.walk(patches_dir)

for root, folders, files in walkthrough:
  if not files: date_list = folders
  elif files: 
    patches_list = files
    break

patches_list = [tif for tif in patches_list if tif.endswith(".tif")]

print(f"[DEBUG] Writing in progress: ", end="")
print("[", end="")
for i, patch in enumerate(patches_list):
  if not (i + 1) % (len(patches_list) // 100): print(".", end="")
  patches_arrays = []

  try:
    output_file = os.path.join(data_cube_dir, f"Data_Cube_{patch[6:-4]}.zarr")

    for date in date_list:
      datetime = np.datetime64(f"{date[:4]}-{date[4:6]}-{date[-2:]}")
      patch_path = os.path.join(patches_dir, date, patch)
      patch_file = rioxarray.open_rasterio(patch_path)
      patch_file = patch_file.assign_coords(time=datetime)
      patches_arrays.append(patch_file)
    
    data_cube = xarray.concat(patches_arrays, dim="time")
    data_cube = data_cube.transpose("band", "time", "y", "x")
    data_cube.name = "data_cube"

    data_cube.to_zarr(
      store=output_file,
      mode="w",
      consolidated=False
    )

  except Exception as e:
    message = f"[ERROR] {e.__class__.__name__} {e} for {output_file}, date {datetime} at {dt.datetime.now()} ."
    print(message)
    with open(cube_log, "a") as file:
      file.write(message + "\n")
  
print("]")

[DEBUG] Writing in progress: [....................................................................................................]
